## Create a Knowledge Base with Semantic Chunking Strategy
#### What will we do in this workshop?
1. Create a Knowledgebase (KB) in the vector database.
2. We will create a data source for the KB. The data source will be the Amazon Science and 10K documents stored in S3.
3. We will ingest the data from S3, use Semantic Chunking to chunk the data, generate vector embeddings, and store the chunks and their corresponding vector embeddings in the KB.
4. We will then ask some questions and query the KB to return some chunks and inspect relevancy score.
<br>Note: We are not sending the query and its chunks to a LLM in this notebook. We will do that in other notebooks.
![We are generating vector embeddings and storing them in a KB in Vector Database](./Semantic_Chunking.png)

## Concept

Semantic chunking analyzes the relationships within a text (using vector embeddings) and creates chunks based on the semantic similarity calculated by the embedding model. Please note that this will result in chunks with varying sizes. This approach preserves the information’s integrity during retrieval, helping to ensure accurate and contextually appropriate results. <br>
<br>![How Semantic Chunking Works](./Semantic_how_it_works.png)

## Benefits

* By focusing on the text’s meaning and context, semantic chunking significantly improves the quality of retrieval. It should be used in scenarios where maintaining the semantic integrity of the text is crucial.

* Although this method is more computationally intensive than fixed-size chunking, it can be beneficial for chunking documents where contextual boundaries aren’t clear—for example, legal documents, technical manuals, documents with too many tables.

## Cost Considerations

* Since Semantic chunking process generates vector embeddings to find chunk boundaries, this will result in increase API calls to an embedding LLM. Thus, expect relatively higher costs than fixed size chunking.
* However, the bulk of these operations happen the first time the documents are processed. In a steady state situation these costs will be incurred only for new documents or changes documents.

In [ ]:
# Import a module with few helper functions. 
# These functions will help us create knowledge base (KB), create data source for KB, and ingest data using semantic chunking to KB.

import importlib
import advanced_rag_utils

# Reload module
importlib.reload(advanced_rag_utils)

# Re-import all functions
from advanced_rag_utils import *

from datetime import datetime, timedelta, UTC

notebook_start_time = datetime.now(UTC)

In [ ]:
# Let's load the variables we saved in the first notebook. We will use these variables
import json
with open("../variables.json", "r") as f:
    variables = json.load(f)

variables

In [ ]:
# Load the dataframe related to costs from a csv file (if it already exists)
df_costs = load_df_from_csv()
df_costs

### 1. Create a Knowledge Base
Let's specify  chunking strategy, name and descripotion for Knowledge Base (KB) and create a KB.

In [ ]:
model_id = "amazon.titan-embed-text-v2:0"
kb_chunking_strategy = "semantic" # ["fixed", "hierarchical", "semantic", "custom"]

In [ ]:
kb_name = f"advanced-rag-workshop-{kb_chunking_strategy}-chunking"

kb_description = "Knowledge base using Amazon OpenSearch Service as a vector store"

kb = create_kb(kb_name, kb_description, kb_chunking_strategy, variables, model_id)

### 2. Create Datasource for Knowledge Base

In [ ]:
data_source_name = f"advanced-rag-example-{kb_chunking_strategy}"

ds_object = create_data_source_for_kb(kb_chunking_strategy, data_source_name, kb, variables)

### 3. Start Ingestion Job for Amazon Bedrock Knowledge base pointing to Amazon OpenSearch

> **Note**: The ingestion process will take approximately 2-3 minutes to complete. During this time, the system is processing your documents by:
> 1. Extracting text from the source files
> 2. Chunking the content according to the defined strategy (Fixed / Semantic / Hierachical / Custom)
> 3. Generating embeddings for each chunk
> 4. Storing the embeddings and associated metadata in a Knowledge Base (KB) in OpenSearch vector database
>
> You'll see status updates as the process progresses. Please wait for the "Ingestion job completed successfully" message before proceeding to the next step.

In [ ]:
from time import sleep
ingestion_start_time = datetime.now(UTC)
create_ingestion_job(kb, ds_object, variables)
ingestion_end_time = datetime.now(UTC)

In [ ]:
time_taken = (ingestion_end_time-ingestion_start_time).total_seconds()
print(f"time taken to ingest into KB = {fmt_n(time_taken)} seconds")

## Embedding LLM Costs
1. Specify model id
2. Specify start and end time
3. Invoke a helper function to query cloud watch
5. Calculate costs (please note that pricing is subject to change per region and over time)

<br>![Embedding LLM Input Token Costs](./Input_token_embedding_llm_costs.png)

In [ ]:
vector_store_embedding_cost = get_bedrock_token_based_cost(model_id, ingestion_start_time, ingestion_end_time)
print(json.dumps(vector_store_embedding_cost, indent=4))

In [ ]:
# Let's add or update the cost binfo to dataframe. 
# This will help us compare the costs from various chunking strategies visually.
new_row = {
    'chunking_algo': kb_chunking_strategy,
    'embedding_seconds': vector_store_embedding_cost['duration in minutes']*60,
    'input_tokens': vector_store_embedding_cost['input_tokens'],
    'invocation_count': vector_store_embedding_cost['invocation_count'],
    'total_token_costs': vector_store_embedding_cost['total token costs']
}
df_costs = update_or_add_row(df_costs, new_row)
df_costs

In [ ]:
# Let's save the df
save_df_to_csv(df_costs)

### 4. Retrieve: Use input query to RETRIEVE chunks from Vector Database
We will use a helper function where you can specify the number of chunks to extract.<br>
The helper function will 1/ generate a vector embedding for the query, 2/ search the vector embedding in the Knowledge Base (KB) vector database, 3/ get the number of chunks specified, 4/ Optionally, you can also specify minimum score for similarity in which case the helper function will get chunks with at least the minimum relevancy.

<b>Warning: After data is ingested into a KB, when you query immediately, the results might be empty because of eventual consistency. If that happens, please wait for a few seconds and then retry.</b>

In [ ]:
# Now let's pick the chunks with some minimum relevance score for the same question.
query = "Who is the CEO, CFO, and CTO of Amazon?"

#specify the number of chunks
n_chunks = 5

#Let's specify a minimum similarity score. We should see less chunks retrieved as compared to the previous invocation.
min_score = 0.30

# get chunks from KB
chunks_from_kb = retrieve_from_kb(query, kb, n_chunks, variables, min_score)

print(json.dumps(chunks_from_kb, indent=2))

# You should see less number of chunks retrieved as compared to the previous cell 
# because of the minimum relevance score.

In [ ]:
# Let's summarize with total chunks, minimum score, maximum score, average score, 
# and lastly the number of chunks with a score more than a specified threshold.
score_threshold = 0.30
score_structure = analyze_chunk_scores_above_threshold(chunks_from_kb, score_threshold)
print(json.dumps(score_structure, indent=4))

### Cost Summary for Running This Notebook
In this notebook, we have used an embedding LLM for two purposes. 
1. Populate a vector store for six PDF files and one CSV file. (7 documents in total)
2. Generate a query embedding.

In [ ]:
import time
time.sleep(5)

# Marking notebook endtime
notebook_end_time = datetime.now(UTC)

In [ ]:
from IPython.display import display, Markdown
from advanced_rag_utils import embedding_cost_report

cost_for_notebook = get_bedrock_token_based_cost(model_id, notebook_start_time, notebook_end_time)

# Your assumptions for your use case:
scenario_number_of_documents = 100000
scenario_number_of_queries =   5000000
 
display(Markdown(embedding_cost_report(vector_store_embedding_cost, cost_for_notebook, scenario_number_of_documents, scenario_number_of_queries)))